In [1]:
import evalml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc

from dotenv import dotenv_values
from evalml.automl import AutoMLSearch
from sklearn.model_selection import KFold

In [2]:
config = dotenv_values('../.env')

In [3]:
train = pd.read_parquet(config["ENGINEERED_DATA"] + "train_fe.parquet")

In [4]:
train.head()

,customer_ID,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,...,D_64_nunique,D_66_count,D_66_first,D_66_last,D_66_nunique,D_68_count,D_68_first,D_68_last,D_68_nunique,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.938469,0.933824,0.024194,0.868580,0.960384,0.934745,0,0.230769,0.832050,...,1,13,-1,-1,1,13,6,6,1,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.929122,0.899820,0.022119,0.861109,0.929122,0.880519,13,7.153846,6.743468,...,1,13,-1,-1,1,13,6,6,1,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.876615,0.878454,0.028911,0.797670,0.904482,0.880875,0,0.000000,0.000000,...,1,13,-1,-1,1,13,6,6,1,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.567442,0.598969,0.020107,0.567442,0.623392,0.621776,9,1.538462,3.017046,...,1,13,-1,-1,1,13,2,3,3,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.936842,0.891679,0.042325,0.805045,0.940382,0.871900,0,0.000000,0.000000,...,1,13,1,1,1,13,6,6,1,0


In [5]:
train[train.columns[1:-1]]

,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,D_39_min,...,D_64_last,D_64_nunique,D_66_count,D_66_first,D_66_last,D_66_nunique,D_68_count,D_68_first,D_68_last,D_68_nunique
0,0.938469,0.933824,0.024194,0.868580,0.960384,0.934745,0,0.230769,0.832050,0,...,0,1,13,-1,-1,1,13,6,6,1
1,0.929122,0.899820,0.022119,0.861109,0.929122,0.880519,13,7.153846,6.743468,0,...,0,1,13,-1,-1,1,13,6,6,1
2,0.876615,0.878454,0.028911,0.797670,0.904482,0.880875,0,0.000000,0.000000,0,...,2,1,13,-1,-1,1,13,6,6,1
3,0.567442,0.598969,0.020107,0.567442,0.623392,0.621776,9,1.538462,3.017046,0,...,0,1,13,-1,-1,1,13,2,3,3
4,0.936842,0.891679,0.042325,0.805045,0.940382,0.871900,0,0.000000,0.000000,0,...,0,1,13,1,1,1,13,6,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,0.859869,0.848750,0.041969,0.730505,0.895553,0.844229,3,3.846154,6.656402,0,...,3,1,13,1,1,1,13,6,6,1
458909,0.863153,0.859327,0.012425,0.831279,0.868121,0.831279,0,2.076923,4.192484,0,...,2,1,13,-1,-1,1,13,6,6,1
458910,0.793724,0.786838,0.014312,0.756983,0.802953,0.800522,18,7.384615,6.212064,0,...,3,1,13,-1,-1,1,13,5,5,1
458911,0.856981,0.804454,0.037442,0.754129,0.856981,0.754129,0,0.923077,2.752621,0,...,3,2,13,-1,-1,1,13,4,3,2


In [6]:
train["target"]

0         0
1         0
2         0
3         0
4         0
         ..
458908    0
458909    0
458910    0
458911    1
458912    0
Name: target, Length: 458913, dtype: int64

In [8]:
FEATURES = train.columns[1:-1]

automl = AutoMLSearch(X_train=train[FEATURES], y_train=train["target"], 
                      problem_type='binary', ensembling=True, patience=100,verbose=True)
automl.search()

Using default limit of max_batches=4.


*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 4 batches for a total of None pipelines. 
Allowed model families: 



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 8.957


C:\Users\Kaleb\anaconda3\envs\kaggle\lib\site-packages\jupyter_client\session.py:716: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



KeyboardInterrupt: 